# Exploration Study
**Objective**: Test which blind exploration regime is optimal. Compare Lévy walk like motion to ballistic motion.

Agents forage for randomly distributed food particles in a 2D environment.
How much food does each strategy consume?

Consider:
- blind
- independent
- non evolving

agents that are randomly placed in the environment.
All agents behave the same. So placing $n$ agents in the environment is the same as randomly initializing one agent $n$ times.

In [1]:
import os
os.chdir(os.path.join(os.getcwd(), "../code"))

from environment import Environment
from simulation import Params
from data_io import initialize_epoch_data, save_simulation_context, save_epoch_data, update_epoch_data, update_meal_timelines
from simulation import Simulation
import numpy as np
from agent import LévyAgent, BallisticAgent
from visualization import animate_single_iteration
from tqdm import tqdm

# Parameter Settings

In [2]:
NUM_FOOD = 100
SIZE = 100
VELOCITY = 1
EAT_RADIUS = 1
NUM_ITERATIONS = 1
POPULATION_SIZE = 20
TOTAL_TIME = 499
DELTA_T = 1

RUNS = 2
ADD_FOOD_MANUALLY = False
WALLS = False

# Set Up

In [3]:
params = Params(
    num_food = NUM_FOOD,
    size = SIZE,
    velocity = VELOCITY,
    eat_radius = EAT_RADIUS,
    iterations_per_epoch = NUM_ITERATIONS,
    num_epochs = 1,
    population_size = POPULATION_SIZE,
    total_time = TOTAL_TIME,
    delta_t = DELTA_T,
    empty = False,
    border_buffer = 2,
    food_buffer = 2,
    perception_radius = EAT_RADIUS
)

environment = Environment(params)

if WALLS:
    environment.add_wall(np.array([0, 0]), np.array([0, params.size]))
    environment.add_wall(np.array([0, 0]), np.array([params.size, 0]))
    environment.add_wall(np.array([params.size, 0]), np.array([params.size, params.size]))
    environment.add_wall(np.array([0, params.size]), np.array([params.size, params.size]))

if ADD_FOOD_MANUALLY:
    environment.custom_food_positioning()
    params.num_food = environment.num_food

folder = 'exploration_study'
save_simulation_context(folder, environment, params)

consumptions_levy = []
consumptions_ballistic = []

# Lévy Agents

In [4]:
def run_lévy():
    population = []
    for _ in range(params.population_size):
        agent = LévyAgent(params)
        population.append(agent)

    data = initialize_epoch_data(params)
    sim = Simulation(params)

    for _ in range(sim.iterations_per_epoch):
        sim.recycle_agents(population, environment, step=0)
        for step in tqdm(range(params.simulation_steps - 1)):
            for agent in population:
                agent.perceive(environment)
                if agent.pending_steps == 0:
                    agent.choose_action()
                agent.pending_steps -= 1
                new_position = agent.position + np.array([np.cos(agent.direction), np.sin(agent.direction)]) * agent.velocity * params.delta_t
                agent.move(new_position, environment)
            sim.store_positions(population, step + 1)
        sim.store_consumed_meals(population)
        update_meal_timelines(data, sim.iteration, population)
        update_epoch_data(data, sim.iteration, sim.trajectory_log, sim.meals_per_iteration)
        sim.iteration += 1

    # save_epoch_data(folder + '/levy_data', data, None, params.num_epochs)
    # animate_single_iteration(params.iterations_per_epoch - 1, environment, params, data, folder, 'levy_mu1', 0, elite_only = False)
    consumptions_levy.append(data['meals'].values.sum())

# Ballistic Agents

In [5]:
def run_ballistic():
    population = []
    for _ in range(params.population_size):
        agent = BallisticAgent(params)
        population.append(agent)

    data = initialize_epoch_data(params)
    sim = Simulation(params)

    for _ in range(sim.iterations_per_epoch):
        sim.recycle_agents(population, environment, step=0)
        for step in tqdm(range(params.simulation_steps - 1)):
            for agent in population:
                agent.perceive(environment)
                new_position = agent.position + np.array([np.cos(agent.direction), np.sin(agent.direction)]) * agent.velocity * params.delta_t
                agent.move(new_position, environment)
            sim.store_positions(population, step + 1)
        sim.store_consumed_meals(population)
        update_meal_timelines(data, sim.iteration, population)
        update_epoch_data(data, sim.iteration, sim.trajectory_log, sim.meals_per_iteration)
        sim.iteration += 1

    # save_epoch_data(folder + '/ballistic_data', data, None, params.num_epochs)
    # animate_single_iteration(params.iterations_per_epoch - 1, environment, params, data, folder, 'ballistic', 0, elite_only = False)
    consumptions_ballistic.append(data['meals'].values.sum())

# Comparison

In [6]:
for _ in range(RUNS):
    run_lévy()
    run_ballistic()

# calculate average consumtions
consumptions_levy = np.array(consumptions_levy)
consumptions_ballistic = np.array(consumptions_ballistic)
print('average number of food particles eaten by Lévy agents over all runs:      ', consumptions_levy.mean())
print('average number of food particles eaten by ballistic agents over all runs: ', consumptions_ballistic.mean())

100%|██████████| 499/499 [00:01<00:00, 272.66it/s]

average number of food particles eaten by Lévy agents over all runs:       99.5
average number of food particles eaten by ballistic agents over all runs:  185.5
